# Variational MPS and iTEBD for Three-body Spin Model

This notebook contains code for two different approaches to obtain ground state of a three-body spin model, namely:
1. variational matrix product state method (variational MPS),
2. infinite-time evolving block decimation method (iTEBD).

The model we are interested in is of the following Hamiltonian:
$$
H = -\sum_j (\sigma_{j-1}^z \sigma_{j}^x \sigma_{j+1}^z + \sigma_j^y).
$$
And we would like to investigate the ground state of this model on a $N=10$ lattice with open boundary condition using 2-site variational MPS method and on an infinite length lattice using 3-site iTEBD method.

# Variational MPS

Firstly, let's give variational MPS a try.

## MPO form of Hamiltonian

To implement variational MPS, we shall construct the matrix product operator (MPO) first.

The MPO for this model is
$$
M = \begin{pmatrix}
I &-\sigma^z &0 &-\sigma^y\\
0 &0 &\sigma^x &0\\
0 &0 &0 &\sigma^z\\
0 &0 &0 &I
\end{pmatrix}.
$$
Let's implement this.

In [1]:
import numpy as np

Sx = np.array([[0, 1], [1, 0]])
Sy = np.array([[0, -1j], [1j, 0]])
Sz = np.array([[1, 0], [0, -1]])
I = np.eye(2)

mpo = np.zeros((4, 2, 4, 2), dtype=complex)
mpo[0, :, 0, :] = I
mpo[0, :, 1, :] = -Sz
mpo[0, :, 3, :] = -Sy
mpo[1, :, 2, :] = Sx
mpo[2, :, 3, :] = Sz
mpo[3, :, 3, :] = I

## State Initialization

The second thing we shall do is to define a matrix product state and initialize it.

In [2]:
# import subroutines for tensor manipulation
import sys
sys.path.append("..")
from sample_code.vMPS_iTEBD import Sub180221 as Sub

def init_mps(n_site, dim_phys, dim_bond):
    mps = [None] * n_site
    for i in range(n_site):
        dim_left = min(dim_phys ** i, dim_phys ** (n_site - i), dim_bond)
        dim_right = min(dim_phys ** (i+1), dim_phys ** (n_site-1-i), dim_bond)
        mps[i] = np.random.rand(dim_left, dim_phys, dim_right)
    
    # canonicalize
    U = np.eye(np.shape(mps[-1])[-1])
    for i in range(n_site - 1, 0, -1):
        U, mps[i] = Sub.Mps_LQP(mps[i], U)
    
    return mps

## Environment Initialization

The third step is to multiply out all the MPOs and sandwitch it with the MPS to obtain the whole tensor.

This step is called environment initialization (i.e. all the tensors in $\bra{\psi}H\ket{\psi}$).

In [3]:
def initH(mpo, mps):
    n_site = len(mps)
    dim_mpo = np.shape(mpo)[0]
    
    H_left = [None] * n_site
    H_right = [None] * n_site
    
    H_left[0] = np.zeros((1, dim_mpo, 1))
    H_left[0][0, 0, 0] = 1
    H_right[-1] = np.zeros((1, dim_mpo, 1))
    H_right[-1][0, -1, 0] = 1
    
    for i in range(n_site - 1, 0, -1):
        H_right[i - 1] = Sub.NCon([H_right[i], mps[i], mpo, np.conj(mps[i])], 
        [[1, 3, 5], [-1, 2, 1], [-2, 2, 3, 4], [-3, 4, 5]])
    
    return H_left, H_right

## Site Update

The final step is to update each two site in an iterative manner and obtain the ground state in covergence.

We implement the two site update precedure first and then carry out the sweeping.

For the two site update precedure, we can generalize it to **arbitrary n sites** update precedure as below.

In [4]:
import scipy.sparse.linalg as LAs

def update_sites(mpo, H_left, H_right, site_tensor_list, reverse=False):
    '''
    Update arbitrary n consecutive sites in the MPS
    '''
    shape_site_tensor_list = list(map(np.shape, site_tensor_list))
    dim_site_tensor_list = list(map(lambda x: x[1], shape_site_tensor_list))
    n_sites = len(shape_site_tensor_list)
    
    # contract and minimize energy
    contract_rule = [[-1, 1, -(n_sites + 3)]] + \
                    [[i, -(n_sites + 3 + i), i + 1, -(1 + i)] for i in range(1, n_sites + 1)] + \
                    [[-(2 * n_sites + 4), n_sites + 1, -(n_sites + 2)]]
    H_eff = Sub.NCon([H_left] + [mpo] * n_sites + [H_right], contract_rule)
    H_eff = Sub.Group(H_eff, [[i for i in range(n_sites + 2)], [i for i in range(n_sites + 2, 2 * n_sites + 4)]])
    eigval, eigvec = LAs.eigsh(H_eff, k=1, which='SA')

    # update tensor
    if not reverse:
        # left to right
        updated_site_tensor = np.reshape(eigvec, [H_left.shape[-1]] + dim_site_tensor_list + [H_right.shape[0]])
        updated_site_tensor_list = [None] * n_sites
        for i in range(n_sites - 1):
            svd_tensor = Sub.Group(updated_site_tensor, [[0, 1], list(range(2, len(updated_site_tensor.shape)))])
            u, s, v = np.linalg.svd(svd_tensor, full_matrices=False)
            dim_trunc = min(len(s), shape_site_tensor_list[i][-1])
            u = u[:, :dim_trunc]; s = s[:dim_trunc]; v = v[:dim_trunc, :]
            updated_site_tensor_list[i] = u.reshape(shape_site_tensor_list[i])
            updated_site_tensor = np.diag(s) @ v
        updated_site_tensor_list[-1] = updated_site_tensor.reshape(shape_site_tensor_list[-1])
    else:
        # right to left
        updated_site_tensor = np.reshape(eigvec, [H_left.shape[0]] + dim_site_tensor_list + [H_right.shape[-1]])
        updated_site_tensor_list = [None] * n_sites
        for i in range(n_sites - 1, 0, -1):
            svd_tensor = Sub.Group(updated_site_tensor, [list(range(len(updated_site_tensor.shape) - 2)), [len(updated_site_tensor.shape) - 2, len(updated_site_tensor.shape) - 1]])
            u, s, v = np.linalg.svd(svd_tensor, full_matrices=False)
            dim_trunc = min(len(s), shape_site_tensor_list[i][0])
            u = u[:, :dim_trunc]; s = s[:dim_trunc]; v = v[:dim_trunc, :]
            updated_site_tensor_list[i] = v.reshape(shape_site_tensor_list[i])
            updated_site_tensor = u @ np.diag(s)
        updated_site_tensor_list[0] = updated_site_tensor.reshape(shape_site_tensor_list[0])
    
    return updated_site_tensor_list, eigval

Now we can implement the sweeping precedure.

Here we update two sites per step.

In [16]:
def sweep(n_iter, mpo, H_left, H_right, mps):
    n_sites = len(mps)
    eig0 = np.zeros(n_sites)
    eig1 = np.zeros(n_sites)
    
    for r in range(n_iter):
        print(f'Iteration {r}:')
    
        for i in range(n_sites - 1):
            mps[i:i+2], eig1[i] = update_sites(mpo, H_left[i], H_right[i+1], mps[i:i+2], reverse=False)
            mps[i], U = Sub.Mps_QR0P(mps[i])
            H_left[i+1] = Sub.NCon([H_left[i], np.conj(mps[i]), mpo, mps[i]], 
                            [[1, 3, 5], [1, 2, -1], [3, 4, -2, 2], [5, 4, -3]])
            mps[i+1] = np.tensordot(U, mps[i+1], (1, 0))
        
        for i in range(n_sites - 1, 1, -1):
            mps[i-1:i+1], eig1[i] = update_sites(mpo, H_left[i-1], H_right[i], mps[i-1:i+1], reverse=True)
            U, mps[i] = Sub.Mps_LQ0P(mps[i])
            H_right[i-1] = Sub.NCon([H_right[i], mps[i], mpo, np.conj(mps[i])], 
                            [[1, 3, 5], [-1, 2, 1], [-2, 2, 3, 4], [-3, 4, 5]])
            mps[i-1] = np.tensordot(mps[i-1], U, (2,0))
        
        print(eig1)
        if (abs(eig1 - eig0) < 1.0e-7).all():
            break
        eig0 = eig1.copy()
    
    print(f'energy per site: {eig1/n_sites}')

    return mps

## Demo

Now let's demonstrate this approach by calculating the ground state energy, magnetization $\langle \sigma^z_i\rangle$ and $\langle \sigma^x_i\rangle$ on our model.

We choose `dim_bond = 4, 6`.

In [17]:
for dim_bond in [4, 6]:
    print(f'dim_bond = {dim_bond}:\n' + '#'*50)
    n_sites = 10; dim_phys = 2
    mpo = mpo
    mps = init_mps(n_sites, dim_phys, dim_bond)
    H_left, H_right = initH(mpo, mps)
    mps = sweep(100, mpo, H_left, H_right, mps)

    # transfer matrix
    transfer_matrix = [None] * n_sites
    transfer_matrix_sz = [None] * n_sites
    transfer_matrix_sx = [None] * n_sites
    for i in range(n_sites):
        transfer_matrix[i] = Sub.Group(Sub.NCon([mps[i], np.conj(mps[i])], [[-1, 1, -2], [-3, 1, -4]]), [[0, 2], [1, 3]])
        transfer_matrix_sz[i] = Sub.Group(Sub.NCon([mps[i], Sz, np.conj(mps[i])], [[-1, 1, -2], [1, 2], [-3, 2, -4]]), [[0, 2], [1, 3]])
        transfer_matrix_sx[i] = Sub.Group(Sub.NCon([mps[i], Sx, np.conj(mps[i])], [[-1, 1, -2], [1, 2], [-3, 2, -4]]), [[0, 2], [1, 3]])

    # calculate observables
    sz = np.zeros(n_sites)
    sx = np.zeros(n_sites)
    for i in range(n_sites):
        transfer_matrix_list = transfer_matrix.copy()
        transfer_matrix_list[i] = transfer_matrix_sz[i]
        sz[i] = np.real(Sub.NCon(transfer_matrix_list, [[-1, 1]] + [[i+1, i+2] for i in range(n_sites-2)] + [[n_sites-1, -2]]).trace())
        transfer_matrix_list = transfer_matrix.copy()
        transfer_matrix_list[i] = transfer_matrix_sx[i]
        sx[i] = np.real(Sub.NCon(transfer_matrix_list, [[-1, 1]] + [[i+1, i+2] for i in range(n_sites-2)] + [[n_sites-1, -2]]).trace())

    print(f'<s^z_i> = {sz}')
    print(f'<s^x_i> = {sx}')


dim_bond = 4:
##################################################
Iteration 0:
[ -4.61777889  -5.65328729 -11.48597565 -11.48639315 -11.48612377
 -11.48353673 -11.48078361 -11.46715414 -11.46679599 -11.46679599]
Iteration 1:
[-11.48597565 -11.48597565 -11.48597513 -11.4863943  -11.48660147
 -11.48666684 -11.4866028  -11.48639636 -11.48597733 -11.48597733]
Iteration 2:
[-11.48597513 -11.48597513 -11.48597502 -11.4863942  -11.48660113
 -11.48666585 -11.48660113 -11.4863942  -11.48597503 -11.48597503]
Iteration 3:
[-11.48597502 -11.48597502 -11.48597502 -11.4863942  -11.48660113
 -11.48666585 -11.48660113 -11.4863942  -11.48597502 -11.48597502]
Iteration 4:
[-11.48597502 -11.48597502 -11.48597502 -11.4863942  -11.48660113
 -11.48666585 -11.48660113 -11.4863942  -11.48597502 -11.48597502]
energy per site: [-1.1485975  -1.1485975  -1.1485975  -1.14863942 -1.14866011 -1.14866659
 -1.14866011 -1.14863942 -1.1485975  -1.1485975 ]
<s^z_i> = [-9.37245777e-14  2.89257748e-13 -3.64948790e-13 -1.131

## Compare with Exact Diagonalization

We can further compare our results with exact diagonalization.

We borrow code from `task1`.

Note that it's open boundary condition!

In [22]:
def readBit(num, n):
    return (num & (1 << n)) >> n

def flipBit(num, n):
    return num ^ (1 << n)

from scipy import sparse

N = 10
length = 2 ** N

# Hamiltonian
HFrom = []
HTo = []
HValue = []

print('Exact Diagonalization:')

for fromBasis in range(length):

    for i in range(N):
        HFrom.append(fromBasis)
        HTo.append(flipBit(fromBasis, i))
        if i == 0 or i == N-1: # open boundary condition
            HValue.append(- (1 - 2*readBit(fromBasis, i))*1j)
        else:
            HValue.append(-(1 - 2*readBit(fromBasis, (i-1)%N)) * (1 - 2*readBit(fromBasis, (i+1)%N)) - (1 - 2*readBit(fromBasis, i))*1j)

Hed = sparse.coo_matrix((HValue, (HTo, HFrom)), shape=(length, length)).toarray()
eigval, eigvec = LAs.eigsh(Hed, k=1, which='SA')
print(f'Energy: {eigval}, Energy per site: {eigval/N}')

Exact Diagonalization:
Energy: [-11.48944207], Energy per site: [-1.14894421]


The ground state energies only differs by $10^{-4}$!